In [29]:
import os
import random
import pandas as pd
from datetime import datetime, timedelta
filepath = '/Users/Peter/Desktop/5000/Data Science/Projects/Python/MusicBox/data/data/data/'

In [26]:
FileName = ['play/all_play.log', 'search/all_search.log', 'down/all_down.log']
possibleFixedDate = [str(date) for date in range(20170419, 20170426)]
print(possibleFixedDate)
churnRate = 0.3
activeRate = 0.15
names = ['user_id', 'phone', 'search_1', 'search_7', 'search_14', 
         'play_1', 'play_7', 'play_14', 'down_1', 'down_7', 'down_14', 'churn']
data = dict()
for name in names:
    data[name] = []

def generate_partial_df(Fixed_Date, data):
    user_info = dict()
    type_dict = {'s':0, 'p':1, 'd':2}
    timePeriods = [1, 7, 14]
    dates = [(datetime.strptime(Fixed_Date, '%Y%m%d') - timedelta(days = timePeriod)).strftime('%Y%m%d') for timePeriod in timePeriods]
    with open('select.log', 'r') as f:
        for line in f:
            line_list = line[:-1].split('\t')
            line_list = [line.replace(" ", "") for line in line_list]
            line_list = [line.replace("\n", "") for line in line_list]
            if line_list[0] not in user_info:
                user_info[line_list[0]] = [line_list[1]] + [0] * 9 + [1]
            if line_list[2] > Fixed_Date:
                user_info[line_list[0]][10] = 0
            else:
                type_index = type_dict[line_list[3]]
                for index, date in enumerate(dates):
                    if line_list[2] >= date:
                        user_info[line_list[0]][type_index * 3 + 1 + index] += 1
    print('user info generate complete')
    for user_id, info in user_info.items():
        data[names[0]].append(user_id)
        for i in range(len(info)):
            data[names[i + 1]].append(info[i])



['20170419', '20170420', '20170421', '20170422', '20170423', '20170424', '20170425']


In [27]:
for FixedDate in possibleFixedDate:
    startDate = (datetime.strptime(FixedDate, '%Y%m%d') - timedelta(weeks = 2)).strftime('%Y%m%d') 
    endDate = (datetime.strptime(FixedDate, '%Y%m%d') + timedelta(weeks = 2)).strftime('%Y%m%d')
    print('start Date is :%s and end Date is :%s' % (startDate, endDate))
    active_users = dict()
    with open(filepath + 'play/all_play.log', 'r') as f:
        for line in f:
            line_list = line.split('\t')
            line_list = [line.replace(" ", "") for line in line_list]
            line_list = [line.replace("\n", "") for line in line_list]
            if (len(line_list) == 3):
                if line_list[2] < FixedDate and line_list[2] >= startDate:
                    if line_list[0] in active_users:
                        active_users[line_list[0]] += 1
                    else:
                        active_users[line_list[0]] = 1
    Active_Users_Before = set()
    for user , count in active_users.items():
        if count > 3:
            Active_Users_Before.add(user)
    print('real active user proportion is :%f' % (float(len(Active_Users_Before)) / float(len(active_users))))
    Active_Users_After = set()
    for name in FileName:
        print('File %s Start' % name)
        with open(filepath + name, 'r') as f:
            for line in f:
                line_list = line.split('\t')
                line_list = [line.replace(" ", "") for line in line_list]
                if (len(line_list) == 3):
                    if line_list[2] >= FixedDate and line_list[2] <= endDate:
                        Active_Users_After.add(line_list[0])
        print('File %s Complete' % name)
    churnUser = set()
    for user in Active_Users_Before:
        if (user not in Active_Users_After):
            churnUser.add(user)
    print('churnUser Number is: %d' % len(churnUser))
    chosenUser = set()
    churnUserNum = 0
    activeUserNum = 0
    for user in Active_Users_Before:
        if (user in churnUser):
            if random.random() < churnRate:
                chosenUser.add(user)
                churnUserNum += 1
        else:
            if random.random() < activeRate:
                chosenUser.add(user)
                activeUserNum += 1
    print('chosenUser Number is: %d' % len(chosenUser))
    print('Active User Before Number is: %d' % len(Active_Users_Before))
    print('Active User After Number is: %d' % len(Active_Users_After))
    print('Active Chosen User Number is :%d' % activeUserNum)
    print('Churn Chosen User Number is :%d' % churnUserNum)
    outputFile = open('select.log', 'w')
    typeName = ['p', 's', 'd']
    for index, name in enumerate(FileName):
        print('File %s Output Start' % name)
        with open(filepath + name, 'r') as f:
            for line in f:
                line_list = line[:-1].split('\t')
                line_list = [line.replace(" ", "") for line in line_list]
                line_list = [line.replace("\n", "") for line in line_list]
                if (len(line_list) == 3 and line_list[0] in chosenUser and line_list[2] >= startDate and line_list[2] <= endDate):
                    line = line_list[0] + '\t' + line_list[1] + '\t' + line_list[2] + '\t' +  typeName[index] + '\n'
                    outputFile.write(line)
        print('File %s Output Complete' % name)

    outputFile.close()
    generate_partial_df(FixedDate, data)

start Date is :20170405 and end Date is :20170503
real active user proportion is :0.884161
File play/all_play.log Start
File play/all_play.log Complete
File search/all_search.log Start
File search/all_search.log Complete
File down/all_down.log Start
File down/all_down.log Complete
churnUser Number is: 83459
chosenUser Number is: 54745
Active User Before Number is: 280044
Active User After Number is: 259993
Active Chosen User Number is :29634
Churn Chosen User Number is :25111
File play/all_play.log Output Start
File play/all_play.log Output Complete
File search/all_search.log Output Start
File search/all_search.log Output Complete
File down/all_down.log Output Start
File down/all_down.log Output Complete
user info generate complete
start Date is :20170406 and end Date is :20170504
real active user proportion is :0.876884
File play/all_play.log Start
File play/all_play.log Complete
File search/all_search.log Start
File search/all_search.log Complete
File down/all_down.log Start
File dow

Active Users are defined as users who plays songs for more than 3 times.

active User Number after and before simple filtering (3 times play records)

In [30]:
df = pd.DataFrame(data = data)
df = df[names]
df.to_csv('generate_sliding.csv', index = False)
print('user info dataframe output complete')

user info dataframe output complete


In [31]:
df_generate = pd.read_csv('generate_sliding.csv')

In [32]:
df_generate.head(10)

,user_id,phone,search_1,search_7,search_14,play_1,play_7,play_14,down_1,down_7,down_14,churn
0,168276196,ar,0,0,0,0,8,159,0,0,0,1
1,169017759,ar,1,6,37,228,322,563,0,1,221,0
2,168494395,ar,4,4,5,7,7,56,0,0,0,0
3,168623761,ar,0,1,15,70,130,305,0,0,3,0
4,168133748,ip,1,4,4,2,50,180,0,2,2,0
5,167594561,ar,0,0,0,0,16,91,0,0,0,1
6,169012502,ar,0,0,21,3,49,221,0,0,6,0
7,169012505,ar,0,5,29,11,38,121,0,2,5,0
8,168180904,ar,0,2,2,0,20,30,0,0,0,1
9,168997209,ar,0,0,0,0,33,143,0,0,0,1


In [33]:
df_generate.shape

(363200, 12)